In [2]:
#cd '/Users/nikhilsingh/Downloads/Dalmia_machine_learning/Nikhil'

In [3]:
import numpy as np 
import pandas as pd
from utility.db_data import connection_db
from utility.datetime_util import create_date_columns
import warnings
warnings.filterwarnings('ignore')
from utility.read_table import read_multiple_tables

In [4]:
host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com"
database="dalmia_digital"
user="postgres"
password="dalmia123"
sql_query = "SELECT * FROM dalmia_sales.sales_data where billing_date > '2021-04-01' and sold_to_party_code between 7000000 and 7099999"

In [5]:
df = connection_db(host= host , database=database, user=user , password=password, sql_query=sql_query)

In [6]:
df.head()

,month,billing_date,sales_region,sales_zone,sales_type,manufacturing_plant,manufacturing_plant_name,plant,plant_name,sold_to_party_code,...,direct_depot,incoterms_inco,mode_of_transport,t_zone,tzone_desc,brand,product,bill_quantity,gross_bill_price,zbill_invoice_bill
0,Apr'23,2023-04-11,North East,NE3,Trade,1340,DCBL - MEGHALAYA,G045,"DCBL,HAILAKANDI",7008946,...,Depot,FTP,Road,7881522201,UZANKUPA,DALMIA,PPC,5.0,49500,49500.0
1,Apr'23,2023-04-11,North East,NE3,Trade,1340,DCBL - MEGHALAYA,G045,"DCBL,HAILAKANDI",7036612,...,Depot,FTP,Road,7881512301,HAILAKANDI,DSP,DSP,2.5,26250,26250.0
2,Apr'23,2023-04-11,North East,NE3,Trade,1340,DCBL - MEGHALAYA,G045,"DCBL,HAILAKANDI",7007501,...,Depot,FPD,Road,7880072306,shillongpatty,DALMIA,PPC,5.0,50500,50500.0
3,Apr'23,2023-04-11,North East,NE3,Trade,1340,DCBL - MEGHALAYA,G045,"DCBL,HAILAKANDI",7024054,...,Depot,FPD,Road,7881642207,KAYAH T E,DALMIA,PPC,5.0,50500,50500.0
4,Apr'23,2023-04-11,North East,NE3,Trade,1340,DCBL - MEGHALAYA,G045,"DCBL,HAILAKANDI",7024054,...,Depot,FPD,Road,7881642207,KAYAH T E,DALMIA,PPC,5.0,50500,50500.0


In [177]:
def return_yoy_df(df):
    df_sales = create_date_columns([df])
    df_sales = df_sales[['sold_to_party_code','Month_Year','gross_bill_price']]
    #df_sales =df_sales[df_sales['sold_to_party_code']!='DONATION']
    df_group_month = df_sales.groupby(['sold_to_party_code','Month_Year']).agg({'gross_bill_price': ['sum']}).reset_index()
    df_group_month.columns = ['Sold To Party Code', 'Month Year' , 'Sum']
    pivot_table = pd.pivot_table(df_group_month, values='Sum',columns='Month Year', index = 'Sold To Party Code')
    return pivot_table
pivot_price = return_yoy_df(df)

In [149]:
pivot_price = pivot_price.merge(unique_sold_to_party['sold_to_party_district_name'], 
                                  left_index=True , right_index=True , how = 'inner')

In [184]:
pivot_price_1['dealer_id'] = pivot_price_1.index

In [176]:
unique_sold_to_party


,sold_to_party_code,sold_to_party_district_name
sold_to_party_code,,
7008946,7008946,CACHAR
7036612,7036612,HAILAKANDI
7007501,7007501,CACHAR
7024054,7024054,CACHAR
7029462,7029462,HAILAKANDI
...,...,...
7038879,7038879,Kanpur Nagar
7039297,7039297,Mainpuri
7038363,7038363,Ayodhya


In [185]:
pivot_price = pivot_price.fillna(0)
pivot_price = pivot_price[pivot_price.index.isin(dealer_sales.index)]

In [156]:
pivot_price_1 = pivot_price[['2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04', '2023-05', '2023-06', '2023-07','sold_to_party_district_name']]

In [158]:
pivot_price_1['1st_half'] = pivot_price_1[['2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01']].sum(axis = 1)
pivot_price_1['2st_half'] = pivot_price_1[['2023-02', '2023-03',
       '2023-04', '2023-05', '2023-06', '2023-07']].sum(axis = 1)

In [159]:
pivot_price_1['diff'] = ((pivot_price_1['2st_half'] - pivot_price_1['1st_half'])/pivot_price_1['1st_half'])*100

In [162]:
pivot_price_1 = pivot_price_1.fillna(0)

In [164]:
pivot_price_1_groupby =  pivot_price_1.groupby('sold_to_party_district_name').agg({'1st_half':'sum','2st_half':'sum'})

In [165]:
pivot_price_1_groupby.columns = ['1st_half','2nd_half']
pivot_price_1_groupby['diff_%'] = pivot_price_1_groupby['2nd_half']-pivot_price_1_groupby['1st_half']
pivot_price_1_groupby['diff_%'] = (pivot_price_1_groupby['diff_%']/pivot_price_1_groupby['1st_half'])*100

In [167]:
pivot_price_1_groupby['sold_to_party_district_name'] = pivot_price_1_groupby.index

In [186]:
pivot_price_2 = pivot_price_1.merge(pivot_price_1_groupby['diff_%'], on ='sold_to_party_district_name')

In [188]:
pivot_price_2

,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,sold_to_party_district_name,1st_half,2st_half,diff,dealer_id,diff_%
0,160000.0,148000.0,148000.0,152000.0,919000.0,547000.0,670500.0,1147000.0,341250.0,832000.0,713450.0,611000.0,BHADRAK,2074000.0,4315200.0,108.061716,7000004,35.126708
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BHADRAK,0.0,0.0,0.000000,7000048,35.126708
2,485350.0,727700.0,588900.0,759050.0,2152402.0,400900.0,413200.0,3167700.0,364750.0,802700.0,1438000.0,801100.0,BHADRAK,5114302.0,6987450.0,36.625682,7000078,35.126708
3,687700.0,1209200.0,779600.0,988300.0,993400.0,1433900.0,1598700.0,2647800.0,1838300.0,1929300.0,1374850.0,1353202.0,BHADRAK,6092100.0,10742152.0,76.329213,7000388,35.126708
4,1571300.0,3654664.0,2864460.0,2485162.0,5012770.0,5175722.0,5469894.0,10086935.0,4080554.0,5021361.0,4137554.0,3209966.0,BHADRAK,20764078.0,32006264.0,54.142476,7000391,35.126708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,KHANDWA,0.0,0.0,0.000000,7035838,NaN
11671,423400.0,220400.0,312400.0,294150.0,397900.0,229200.0,447820.0,211400.0,113000.0,249200.0,229700.0,0.0,YADADRI BHUVANAGIRI,1877450.0,1251120.0,-33.360675,7035903,-22.335043
11672,474600.0,679600.0,538680.0,415160.0,568760.0,730450.0,677360.0,453480.0,576050.0,742500.0,220100.0,183750.0,YADADRI BHUVANAGIRI,3407250.0,2853240.0,-16.259740,7035936,-22.335043
11673,1567700.0,770100.0,1628600.0,1611600.0,1584400.0,965600.0,4872200.0,2580600.0,362250.0,0.0,0.0,0.0,PALGHAR,8128000.0,7815050.0,-3.850271,7036011,-3.850271


In [261]:
pivot_price_3 = pivot_price[['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03']]

In [265]:
def dealer_size(var):
    if var<=10000000:
        return 'Small'
    if var>10000000 and var<=50000000:
        return 'Medium'
    if var>50000000:
        return 'Large'
pivot_price_3['total_sales'] = pivot_price_3.sum(axis = 1)
pivot_price_3['dealer_size'] = pivot_price_3['total_sales'].apply(dealer_size)

In [266]:
pivot_price_3

Month Year,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,total_sales,dealer_size
Sold To Party Code,,,,,,,,,,,,,,
7000004,749300.0,934900.0,283500.0,178200.0,160000.0,148000.0,148000.0,152000.0,919000.0,547000.0,670500.0,1147000.0,18112200.0,Medium
7000010,197500.0,139400.0,0.0,264500.0,96000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2092200.0,Small
7000015,219900.0,608900.0,788750.0,327250.0,431000.0,331750.0,545102.0,204600.0,0.0,0.0,0.0,0.0,10371756.0,Medium
7000019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Small
7000025,484500.0,812000.0,1288000.0,1400000.0,600000.0,724000.0,355200.0,824240.0,547800.0,1235800.0,1017400.0,1274200.0,31689420.0,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,232800.0,1296330.0,726990.0,635150.0,598850.0,567800.0,272900.0,558000.0,54300.0,196500.0,459600.0,556000.0,18465660.0,Medium
7036281,81800.0,45280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,381240.0,Small
7036287,207500.0,721200.0,1379400.0,812400.0,694100.0,746600.0,799400.0,896900.0,1027000.0,700000.0,803700.0,1072800.0,29583000.0,Medium


In [271]:
#dealer_sales_5.drop('Dealer_size',axis =1 ,inplace = True)
dealer_sales_5 = dealer_sales_5.merge(pivot_price_3['dealer_size'],left_index=True , right_index=True , how = 'inner' )

In [272]:
dealer_sales_5

,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,...,2023-07,28,sold_to_party_district_name,1st_half,2st_half,diff,dealer_id,diff_%,total_sales,dealer_size
7000004,20.0,20.0,20.00,20.0,122.5,75.0,90.00,155.00,46.0,111.0,...,80.0,Active,BHADRAK,277.50,578.00,108.288288,7000004,35.631560,855.5,Medium
7000010,12.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,24.0,...,24.0,Active,CUTTACK,12.00,60.00,400.000000,7000010,23.801064,72.0,Small
7000015,54.5,45.0,74.00,27.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,Inactive,PURI,200.50,0.00,-100.000000,7000015,16.091194,200.5,Medium
7000025,75.0,95.0,48.00,108.8,73.0,167.0,137.00,173.00,158.0,132.0,...,147.0,Active,PURI,566.80,856.00,51.023289,7000025,16.091194,1422.8,Medium
7000027,513.0,5.0,171.95,207.5,92.0,68.5,35.00,144.95,0.0,0.0,...,0.0,Inactive,BARGARH,1057.95,179.95,-82.990690,7000027,-10.631377,1237.9,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036277,39.0,75.0,26.00,31.0,11.0,72.0,103.00,124.00,77.0,73.0,...,90.0,Active,KEONJHAR,254.00,492.00,93.700787,7036277,19.238393,746.0,Medium
7036279,67.0,65.0,30.00,61.0,6.0,22.5,54.00,70.00,17.0,14.0,...,7.0,Active,Kallakurichi,251.50,167.00,-33.598410,7036279,54.430631,418.5,Medium
7036287,89.0,104.0,111.00,122.0,137.0,100.0,111.00,146.00,200.0,207.0,...,151.0,Active,JAGATSINGHPUR,663.00,1012.00,52.639517,7036287,36.292578,1675.0,Medium
7036291,62.5,75.0,60.00,96.0,108.0,132.0,168.00,178.00,180.0,236.5,...,108.0,Active,Nagpur,533.50,1040.50,95.032802,7036291,38.950880,1574.0,Medium


In [190]:
pivot_price_2.to_csv('distt_wise_price.csv')

In [8]:
pivot = pivot.fillna(0)

In [11]:
def count_zeros_before_first_nonzero(row):
    count = 0
    for value in row:
        if value == 0:
            count += 1
        else:
            break 
    return count
pivot['month_new_dealer'] = pivot.apply(count_zeros_before_first_nonzero , axis = 1)

In [12]:
dealer_sales = pivot[pivot['month_new_dealer']<=12] 

In [13]:
dealer_sales

Month Year,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,month_new_dealer
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,20.0,122.5,75.0,90.00,155.00,46.0,111.0,96.0,80.0,0
7000010,57.0,37.0,0.0,24.0,30.0,0.0,41.0,24.0,25.0,24.0,...,0.0,0.0,0.0,0.00,0.00,0.0,24.0,12.0,24.0,0
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,27.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0
7000019,12.0,50.0,133.0,104.0,165.0,123.5,0.0,7.5,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0
7000025,98.0,108.5,87.0,105.0,107.5,102.0,107.0,117.0,60.0,85.0,...,108.8,73.0,167.0,137.00,173.00,158.0,132.0,109.0,147.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.0,6.0,22.5,54.00,70.00,17.0,14.0,5.0,7.0,12
7036281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,12
7036287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,122.0,137.0,100.0,111.00,146.00,200.0,207.0,197.0,151.0,12


In [14]:
def count_trailing_zeros_before_end_index(df, end_index):
    counts = []
    for row in df.itertuples(index=False):
        count = 0
        for i in range(end_index - 1, -1, -1):
            if row[i] == 0:
                count += 1
            else:
                break
        counts.append(count)
    return counts

result_matrix = pd.DataFrame(index = dealer_sales.index)
for i in range(len(dealer_sales.columns)):
    string = str(i)
    result_matrix[string] = count_trailing_zeros_before_end_index(dealer_sales, i)
result_matrix.drop('0', axis = 1 , inplace = True)
#Status of each dealer 
def inactive(value):
    if value == 0: 
        return 'Active'
    elif value == 1: 
        return 'Active_1N'
    elif value == 2: 
        return 'Active_2N'
    elif value == 3: 
        return 'Active_3N'
    elif value>3 and value <=12:
        return 'Inactive'
    else: 
        return 'Churn'
    
for i in result_matrix.columns:
    result_matrix[i] = result_matrix[i].apply(inactive)


In [15]:
dealer_sales = dealer_sales.merge(result_matrix['28'], left_index = True, right_index = True, how = 'inner')
dealer_sales

,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,month_new_dealer,28
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,122.5,75.0,90.00,155.00,46.0,111.0,96.0,80.0,0,Active
7000010,57.0,37.0,0.0,24.0,30.0,0.0,41.0,24.0,25.0,24.0,...,0.0,0.0,0.00,0.00,0.0,24.0,12.0,24.0,0,Active
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0,Inactive
7000019,12.0,50.0,133.0,104.0,165.0,123.5,0.0,7.5,0.0,0.0,...,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0,Churn
7000025,98.0,108.5,87.0,105.0,107.5,102.0,107.0,117.0,60.0,85.0,...,73.0,167.0,137.00,173.00,158.0,132.0,109.0,147.0,0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,22.5,54.00,70.00,17.0,14.0,5.0,7.0,12,Active
7036281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,12,Churn
7036287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,137.0,100.0,111.00,146.00,200.0,207.0,197.0,151.0,12,Active


In [16]:
dealer_sales.drop('month_new_dealer', axis = 1 , inplace = True)

In [17]:
dealer_sales

,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,28
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,20.0,122.5,75.0,90.00,155.00,46.0,111.0,96.0,80.0,Active
7000010,57.0,37.0,0.0,24.0,30.0,0.0,41.0,24.0,25.0,24.0,...,0.0,0.0,0.0,0.00,0.00,0.0,24.0,12.0,24.0,Active
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,27.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Inactive
7000019,12.0,50.0,133.0,104.0,165.0,123.5,0.0,7.5,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Churn
7000025,98.0,108.5,87.0,105.0,107.5,102.0,107.0,117.0,60.0,85.0,...,108.8,73.0,167.0,137.00,173.00,158.0,132.0,109.0,147.0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.0,6.0,22.5,54.00,70.00,17.0,14.0,5.0,7.0,Active
7036281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Churn
7036287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,122.0,137.0,100.0,111.00,146.00,200.0,207.0,197.0,151.0,Active


In [18]:
district = df[['sold_to_party_code','sold_to_party_district_name']]
unique_sold_to_party = district.drop_duplicates(subset=['sold_to_party_code'])

In [19]:
unique_sold_to_party.index = unique_sold_to_party['sold_to_party_code']

In [20]:
dealer_sales = dealer_sales.merge(unique_sold_to_party['sold_to_party_district_name'], 
                                  left_index=True , right_index=True , how = 'inner')

In [21]:
dealer_sales.columns

Index(['2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09',
       '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03',
       '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04', '2023-05', '2023-06', '2023-07', '28',
       'sold_to_party_district_name'],
      dtype='object')

In [126]:
dealer_sales_1 = dealer_sales[[ '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04', '2023-05', '2023-06', '2023-07', '28',
       'sold_to_party_district_name']].copy()

In [127]:
dealer_sales_1

,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,28,sold_to_party_district_name
7000004,20.0,20.0,20.0,20.0,122.5,75.0,90.00,155.00,46.0,111.0,96.0,80.0,Active,BHADRAK
7000010,12.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,24.0,12.0,24.0,Active,CUTTACK
7000015,54.5,45.0,74.0,27.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Inactive,PURI
7000019,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Churn,PURI
7000025,75.0,95.0,48.0,108.8,73.0,167.0,137.00,173.00,158.0,132.0,109.0,147.0,Active,PURI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,67.0,65.0,30.0,61.0,6.0,22.5,54.00,70.00,17.0,14.0,5.0,7.0,Active,Kallakurichi
7036281,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Churn,PURBA BARDHAMAN
7036287,89.0,104.0,111.0,122.0,137.0,100.0,111.00,146.00,200.0,207.0,197.0,151.0,Active,JAGATSINGHPUR
7036291,62.5,75.0,60.0,96.0,108.0,132.0,168.00,178.00,180.0,236.5,170.0,108.0,Active,Nagpur


In [24]:
distt_groupby = dealer_sales_1.groupby(['sold_to_party_district_name','28']).agg({'28':['count']}).reset_index()

In [25]:
distt_groupby.columns = ['Distt_name' , 'State' , 'Count']

In [38]:
distt_groupby.to_csv('distt_wise_state.csv')

In [26]:
dealer_sales_1.columns

Index(['2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01',
       '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '28',
       'sold_to_party_district_name'],
      dtype='object')

In [27]:
dealer_sales_2 = dealer_sales_1[dealer_sales_1['28'].isin(['Active','Active_1N','Active_2N'])]

In [28]:
dealer_sales_2['1-6_sum'] = dealer_sales_2[['2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01']].sum(axis =1)
dealer_sales_2['7-12_sum'] = dealer_sales_2[['2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07']].sum(axis =1)

In [29]:
dealer_sales_2['6_month_diff'] = dealer_sales_2['7-12_sum'] - dealer_sales_2['1-6_sum']

In [30]:
dealer_sales_2

,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,28,sold_to_party_district_name,1-6_sum,7-12_sum,6_month_diff
7000004,20.0,20.0,20.0,20.0,122.5,75.0,90.00,155.00,46.0,111.0,96.0,80.0,Active,BHADRAK,277.5,578.0,300.5
7000010,12.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,24.0,12.0,24.0,Active,CUTTACK,12.0,60.0,48.0
7000025,75.0,95.0,48.0,108.8,73.0,167.0,137.00,173.00,158.0,132.0,109.0,147.0,Active,PURI,566.8,856.0,289.2
7000029,60.0,60.0,60.0,88.0,113.0,60.0,88.00,185.00,30.0,60.0,55.0,90.0,Active,KANDHAMAL,441.0,508.0,67.0
7000032,405.0,435.0,410.0,421.0,445.0,471.0,470.00,959.50,424.0,460.0,495.0,517.0,Active,BALASORE,2587.0,3325.5,738.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036277,39.0,75.0,26.0,31.0,11.0,72.0,103.00,124.00,77.0,73.0,25.0,90.0,Active,KEONJHAR,254.0,492.0,238.0
7036279,67.0,65.0,30.0,61.0,6.0,22.5,54.00,70.00,17.0,14.0,5.0,7.0,Active,Kallakurichi,251.5,167.0,-84.5
7036287,89.0,104.0,111.0,122.0,137.0,100.0,111.00,146.00,200.0,207.0,197.0,151.0,Active,JAGATSINGHPUR,663.0,1012.0,349.0
7036291,62.5,75.0,60.0,96.0,108.0,132.0,168.00,178.00,180.0,236.5,170.0,108.0,Active,Nagpur,533.5,1040.5,507.0


In [31]:
dealer_sales_2.groupby('sold_to_party_district_name').agg({'6_month_diff':['mean']}).reset_index().to_csv('distt_6m_change.csv')

#### Frequency Order Combinations

In [205]:
def return_yoy_df(df):
    df_sales = create_date_columns([df])
    df_sales = df_sales[['sold_to_party_code','Month_Year','bill_quantity']]
    #df_sales =df_sales[df_sales['sold_to_party_code']!='DONATION']
    df_group_month = df_sales.groupby(['sold_to_party_code','Month_Year']).agg({'Month_Year': ['count']}).reset_index()
    df_group_month.columns = ['Sold To Party Code', 'Month Year' , 'Sum']
    pivot_table = pd.pivot_table(df_group_month, values='Sum',columns='Month Year', index = 'Sold To Party Code')
    return pivot_table
pivot_freq = return_yoy_df(df)

In [206]:
pivot_freq = pivot_freq.fillna(0)


In [207]:
pivot_freq = pivot_freq[pivot_freq.index.isin(dealer_sales_1.index)]

In [208]:
pivot_freq

Month Year,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07
Sold To Party Code,,,,,,,,,,,,,,,,,,,,,
7000004,22.0,5.0,15.0,9.0,7.0,6.0,6.0,5.0,9.0,4.0,...,1.0,1.0,8.0,8.0,5.0,9.0,6.0,10.0,12.0,8.0
7000010,5.0,3.0,0.0,2.0,3.0,0.0,3.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0
7000015,1.0,2.0,4.0,7.0,17.0,14.0,9.0,13.0,7.0,5.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7000019,1.0,2.0,7.0,10.0,17.0,11.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7000025,8.0,10.0,11.0,7.0,6.0,6.0,6.0,5.0,3.0,4.0,...,4.0,8.0,5.0,12.0,6.0,10.0,11.0,12.0,8.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,17.0,2.0,5.0,14.0,14.0,3.0,3.0,1.0,2.0
7036281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7036287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,7.0,9.0,4.0,7.0,10.0,18.0,20.0,19.0,13.0


In [37]:
df_dealer_size = pd.read_excel('data/dealer_list_segement.xlsx')

In [41]:
df_dealer_size.index = df_dealer_size['Dealer_ID']

In [42]:
df_dealer_size

,Dealer_ID,Dealer_size,Level_change,Slope_change,Total_sale_yearly,District_name,Sales_region,ASO_ID,State_m-1,State_m,State_m+1,State_m+2
Dealer_ID,,,,,,,,,,,,
7000004,7000004,Large,Same,Same,784.50,BHADRAK,East,EMP1031614,Active,Active,Active,Active
7000010,7000010,Small,Fall,Same,226.80,CUTTACK,East,EMP0008569,Active,Active_1N,Active_2N,Active_3N
7000015,7000015,Small,Same,High Fall,625.00,PURI,East,EMP1033701,Active,Active_1N,Active_2N,Active_3N
7000022,7000022,Small,Same,Same,155.50,KHURDA,East,EMP0006757,Active,Active_1N,Active,Active
7000025,7000025,Large,Rise,Same,1372.80,PURI,East,EMP0009813,Active,Active,Active,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
7036491,7036491,Small,High Rise,Rise,902.30,NADIA,East,EMP1026780,Active,Active,Active,Active
7036499,7036499,Large,Same,High Rise,1300.70,Maharajganj,Central,EMP1035371,Active,Active,Active,Active
7036501,7036501,Large,High Rise,Same,3152.00,Bangalore,South,EMP1037151,Active,Active,Active,Active


In [43]:
dealer_sales_4 = dealer_sales.merge(df_dealer_size['Dealer_size'], left_index = True, right_index=True , how = 'left')

In [44]:
dealer_sales_4 = dealer_sales_4.dropna()

In [45]:
dealer_sales_4

,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,28,sold_to_party_district_name,Dealer_size
7000004,255.5,85.0,207.00,102.0,73.50,74.5,43.0,25.0,76.0,53.00,...,75.0,90.00,155.00,46.0,111.0,96.0,80.0,Active,BHADRAK,Large
7000010,57.0,37.0,0.00,24.0,30.00,0.0,41.0,24.0,25.0,24.00,...,0.0,0.00,0.00,0.0,24.0,12.0,24.0,Active,CUTTACK,Small
7000015,4.0,50.0,63.00,62.0,91.00,68.5,70.5,77.0,50.0,44.50,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Inactive,PURI,Small
7000025,98.0,108.5,87.00,105.0,107.50,102.0,107.0,117.0,60.0,85.00,...,167.0,137.00,173.00,158.0,132.0,109.0,147.0,Active,PURI,Large
7000027,602.5,523.5,816.25,228.0,628.75,1002.5,929.0,391.5,1235.0,688.35,...,68.5,35.00,144.95,0.0,0.0,0.0,0.0,Inactive,BARGARH,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036277,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,72.0,103.00,124.00,77.0,73.0,25.0,90.0,Active,KEONJHAR,Small
7036279,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,22.5,54.00,70.00,17.0,14.0,5.0,7.0,Active,Kallakurichi,Small
7036287,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,100.0,111.00,146.00,200.0,207.0,197.0,151.0,Active,JAGATSINGHPUR,Medium
7036291,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,132.0,168.00,178.00,180.0,236.5,170.0,108.0,Active,Nagpur,Medium


In [46]:
dealer_sales.columns

Index(['2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09',
       '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03',
       '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04', '2023-05', '2023-06', '2023-07', '28',
       'sold_to_party_district_name'],
      dtype='object')

In [277]:
dealer_sales_4= dealer_sales[['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03', 'sold_to_party_district_name']]

In [279]:
dealer_sales_4 = dealer_sales_4.merge(dealer_sales_5['dealer_size'],left_index = True , right_index=True , how = 'inner')

In [280]:
dealer_sales_4

,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,sold_to_party_district_name,dealer_size
7000004,91.0,114.00,35.00,22.00,20.0,20.0,20.00,20.0,122.5,75.0,90.00,155.00,BHADRAK,Medium
7000010,25.0,17.00,0.00,33.80,12.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,CUTTACK,Small
7000015,27.0,75.50,100.00,42.00,54.5,45.0,74.00,27.0,0.0,0.0,0.00,0.00,PURI,Medium
7000025,60.0,100.00,161.00,175.00,75.0,95.0,48.00,108.8,73.0,167.0,137.00,173.00,PURI,Medium
7000027,276.5,696.25,1222.15,226.25,513.0,5.0,171.95,207.5,92.0,68.5,35.00,144.95,BARGARH,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036277,32.0,169.00,104.00,75.00,39.0,75.0,26.00,31.0,11.0,72.0,103.00,124.00,KEONJHAR,Medium
7036279,24.0,144.00,86.00,74.00,67.0,65.0,30.00,61.0,6.0,22.5,54.00,70.00,Kallakurichi,Medium
7036287,25.0,90.00,174.00,104.00,89.0,104.0,111.00,122.0,137.0,100.0,111.00,146.00,JAGATSINGHPUR,Medium
7036291,12.0,250.00,264.00,109.50,62.5,75.0,60.00,96.0,108.0,132.0,168.00,178.00,Nagpur,Medium


In [281]:
dealer_sales_4['total_sales'] = dealer_sales_4[['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03']].sum(axis = 1)

In [283]:
dealer_sales_4_groupby = dealer_sales_4.groupby(['sold_to_party_district_name',
                                                 'dealer_size']).agg({'total_sales':['sum', 'count']}).reset_index()

In [284]:
dealer_sales_4_groupby.columns = ['District_name','Dealer_size','total_Mar_23','count']

In [285]:
dealer_sales_4_groupby

,District_name,Dealer_size,total_Mar_23,count
0,24 PRAGANA NORTH,Large,82612.25,14
1,24 PRAGANA NORTH,Medium,30039.55,23
2,24 PRAGANA NORTH,Small,1810.50,16
3,24 PRAGANA SOUTH,Large,132967.35,16
4,24 PRAGANA SOUTH,Medium,18097.35,17
...,...,...,...,...
939,YADGIR,Medium,506.00,1
940,YADGIR,Small,1452.00,7
941,Yavatmal,Large,3154.00,1
942,Yavatmal,Medium,8026.00,8


In [55]:
95052/(30*12)

264.03333333333336

In [286]:
dealer_sales_4

,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,sold_to_party_district_name,dealer_size,total_sales
7000004,91.0,114.00,35.00,22.00,20.0,20.0,20.00,20.0,122.5,75.0,90.00,155.00,BHADRAK,Medium,784.50
7000010,25.0,17.00,0.00,33.80,12.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,CUTTACK,Small,87.80
7000015,27.0,75.50,100.00,42.00,54.5,45.0,74.00,27.0,0.0,0.0,0.00,0.00,PURI,Medium,445.00
7000025,60.0,100.00,161.00,175.00,75.0,95.0,48.00,108.8,73.0,167.0,137.00,173.00,PURI,Medium,1372.80
7000027,276.5,696.25,1222.15,226.25,513.0,5.0,171.95,207.5,92.0,68.5,35.00,144.95,BARGARH,Large,3659.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036277,32.0,169.00,104.00,75.00,39.0,75.0,26.00,31.0,11.0,72.0,103.00,124.00,KEONJHAR,Medium,861.00
7036279,24.0,144.00,86.00,74.00,67.0,65.0,30.00,61.0,6.0,22.5,54.00,70.00,Kallakurichi,Medium,703.50
7036287,25.0,90.00,174.00,104.00,89.0,104.0,111.00,122.0,137.0,100.0,111.00,146.00,JAGATSINGHPUR,Medium,1313.00
7036291,12.0,250.00,264.00,109.50,62.5,75.0,60.00,96.0,108.0,132.0,168.00,178.00,Nagpur,Medium,1515.00


In [287]:
dealer_sales_4_groupby.to_csv('District_wise_dealer_size_ratio.csv')

In [288]:
discount_df = pd.read_csv('data/discount_pivot.csv')
discount_df.index = discount_df['Sold To Party Code']
discount_df.drop('Sold To Party Code', axis = 1 , inplace =True)

In [289]:
discount_df = discount_df.merge(dealer_sales_4['sold_to_party_district_name'], left_index=True , right_index=True , how = 'inner')

In [290]:
discount_df['total_discount'] = discount_df[['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03']].sum(axis = 1)

In [291]:
discount_df = discount_df.merge(dealer_sales_4['total_sales'], left_index=True , right_index=True , how = 'inner')

In [292]:
discount_df['total_sales'] = discount_df['total_sales']*20

In [293]:
discount_df['discount_per_bag'] = discount_df['total_discount']/discount_df['total_sales']

In [295]:
discount_df = discount_df.merge(dealer_sales_4['dealer_size'], left_index = True , right_index=True , how = 'inner')

In [296]:
discount_df_groupby = discount_df.groupby(['sold_to_party_district_name' , 'dealer_size']).agg({'discount_per_bag':'mean'}).reset_index()

In [298]:
discount_df_groupby.to_csv('distt_wise_discount.csv')

In [297]:
discount_df_groupby

,sold_to_party_district_name,dealer_size,discount_per_bag
0,24 PRAGANA NORTH,Large,31.716303
1,24 PRAGANA NORTH,Medium,29.926030
2,24 PRAGANA NORTH,Small,24.617131
3,24 PRAGANA SOUTH,Large,36.610205
4,24 PRAGANA SOUTH,Medium,28.571556
...,...,...,...
879,Wokha,Large,36.372854
880,YADADRI BHUVANAGIRI,Medium,65.463239
881,YADADRI BHUVANAGIRI,Small,62.605770
882,YADGIR,Medium,49.861717


In [128]:
dealer_sales_1['1st_half'] = dealer_sales_1[['2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01']].sum(axis=1)
dealer_sales_1['2st_half'] = dealer_sales_1[['2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07']].sum(axis=1)

In [129]:
dealer_sales_1['diff'] = ((dealer_sales_1['2st_half'] - dealer_sales_1['1st_half'])/dealer_sales_1['1st_half'])*100

In [130]:
dealer_sales_1 = dealer_sales_1.fillna(0)

In [131]:
dealer_sales_1

,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,28,sold_to_party_district_name,1st_half,2st_half,diff
7000004,20.0,20.0,20.0,20.0,122.5,75.0,90.00,155.00,46.0,111.0,96.0,80.0,Active,BHADRAK,277.5,578.0,108.288288
7000010,12.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,24.0,12.0,24.0,Active,CUTTACK,12.0,60.0,400.000000
7000015,54.5,45.0,74.0,27.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Inactive,PURI,200.5,0.0,-100.000000
7000019,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Churn,PURI,0.0,0.0,0.000000
7000025,75.0,95.0,48.0,108.8,73.0,167.0,137.00,173.00,158.0,132.0,109.0,147.0,Active,PURI,566.8,856.0,51.023289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036279,67.0,65.0,30.0,61.0,6.0,22.5,54.00,70.00,17.0,14.0,5.0,7.0,Active,Kallakurichi,251.5,167.0,-33.598410
7036281,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,Churn,PURBA BARDHAMAN,0.0,0.0,0.000000
7036287,89.0,104.0,111.0,122.0,137.0,100.0,111.00,146.00,200.0,207.0,197.0,151.0,Active,JAGATSINGHPUR,663.0,1012.0,52.639517
7036291,62.5,75.0,60.0,96.0,108.0,132.0,168.00,178.00,180.0,236.5,170.0,108.0,Active,Nagpur,533.5,1040.5,95.032802


In [132]:
dealer_sales_1st_groupby = dealer_sales_1.groupby('sold_to_party_district_name').agg({'1st_half':'sum','2st_half':'sum'})

In [133]:
dealer_sales_1st_groupby.columns = ['1st_half','2nd_half']

In [134]:
dealer_sales_1st_groupby['diff_%'] = dealer_sales_1st_groupby['2nd_half']-dealer_sales_1st_groupby['1st_half']

In [135]:
dealer_sales_1st_groupby['diff_%']=(dealer_sales_1st_groupby['diff_%']/dealer_sales_1st_groupby['1st_half'])*100

In [136]:
dealer_sales_1st_groupby['sold_to_party_district_name'] = dealer_sales_1st_groupby.index

In [191]:
dealer_sales_1['dealer_id'] = dealer_sales_1.index

In [196]:
dealer_sales_5 = dealer_sales_1.merge(dealer_sales_1st_groupby['diff_%'], 
                                       on='sold_to_party_district_name', how = 'left')

In [197]:
dealer_sales_5.index = dealer_sales_5['dealer_id']

In [198]:
dealer_sales_5 = dealer_sales_5.merge(df_dealer_size['Dealer_size'], left_index=True , right_index=True , how = 'inner')

In [301]:
dealer_sales_5.to_csv('Distt_wise_growth.csv')

In [300]:
dealer_sales_5

,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,...,2023-07,28,sold_to_party_district_name,1st_half,2st_half,diff,dealer_id,diff_%,total_sales,dealer_size
7000004,20.0,20.0,20.00,20.0,122.5,75.0,90.00,155.00,46.0,111.0,...,80.0,Active,BHADRAK,277.50,578.00,108.288288,7000004,35.631560,855.5,Medium
7000010,12.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,24.0,...,24.0,Active,CUTTACK,12.00,60.00,400.000000,7000010,23.801064,72.0,Small
7000015,54.5,45.0,74.00,27.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,Inactive,PURI,200.50,0.00,-100.000000,7000015,16.091194,200.5,Medium
7000025,75.0,95.0,48.00,108.8,73.0,167.0,137.00,173.00,158.0,132.0,...,147.0,Active,PURI,566.80,856.00,51.023289,7000025,16.091194,1422.8,Medium
7000027,513.0,5.0,171.95,207.5,92.0,68.5,35.00,144.95,0.0,0.0,...,0.0,Inactive,BARGARH,1057.95,179.95,-82.990690,7000027,-10.631377,1237.9,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036277,39.0,75.0,26.00,31.0,11.0,72.0,103.00,124.00,77.0,73.0,...,90.0,Active,KEONJHAR,254.00,492.00,93.700787,7036277,19.238393,746.0,Medium
7036279,67.0,65.0,30.00,61.0,6.0,22.5,54.00,70.00,17.0,14.0,...,7.0,Active,Kallakurichi,251.50,167.00,-33.598410,7036279,54.430631,418.5,Medium
7036287,89.0,104.0,111.00,122.0,137.0,100.0,111.00,146.00,200.0,207.0,...,151.0,Active,JAGATSINGHPUR,663.00,1012.00,52.639517,7036287,36.292578,1675.0,Medium
7036291,62.5,75.0,60.00,96.0,108.0,132.0,168.00,178.00,180.0,236.5,...,108.0,Active,Nagpur,533.50,1040.50,95.032802,7036291,38.950880,1574.0,Medium


In [337]:
pivot_freq_1 = pivot_freq[['2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04', '2023-05', '2023-06', '2023-07']]

In [338]:
pivot_freq_1['total_orders'] = pivot_freq_1.sum(axis = 1)

In [339]:
pivot_freq_1 = pivot_freq_1.merge(dealer_sales_5['dealer_size'], left_index=True , right_index=True , how = 'inner')

In [340]:
pivot_freq_1[pivot_freq_1['total_orders']>pivot_freq_1['total_orders'].max()-7180]['dealer_size'].value_counts()

Medium    2942
Large     1813
Small      590
Name: dealer_size, dtype: int64

In [341]:
pivot_freq_1['total_orders'].max()

7228.0

In [342]:
def freq_count(var):
    if var<=20:
        return 'L'
    if var>20 and var<=200:
        return 'M'
    if var>200:
        return 'H'
pivot_freq_1['total_orders'] = pivot_freq_1['total_orders'].apply(freq_count)

In [343]:
pivot_freq_1['total_orders'].value_counts()

M    4695
L    2314
H    2271
Name: total_orders, dtype: int64

In [383]:
dealer_sales_5['total_sales']=dealer_sales_5['1st_half']+dealer_sales_5['2st_half']
def order_size(var):
    if var<=400:
        return 'L'
    if var>400 and var<=3000:
        return 'M'
    if var>3000:
        return 'H'
dealer_sales_5['total_sales'] = dealer_sales_5['total_sales'].apply(order_size)

In [384]:
dealer_sales_5['dealer_size'].value_counts()

Small     3872
Medium    3589
Large     1819
Name: dealer_size, dtype: int64

In [385]:
dealer_sales_5['total_sales'].value_counts()

L    4111
M    3929
H    1240
Name: total_sales, dtype: int64

In [387]:
dealer_sales_6 = dealer_sales_5.merge(pivot_freq_1['total_orders'], left_index = True , right_index = True , how = 'inner')

In [389]:
dealer_sales_6.to_csv('sales_order_frequency.csv')

In [390]:
dealer_sales_6

,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,...,28,sold_to_party_district_name,1st_half,2st_half,diff,dealer_id,diff_%,total_sales,dealer_size,total_orders
7000004,20.0,20.0,20.00,20.0,122.5,75.0,90.00,155.00,46.0,111.0,...,Active,BHADRAK,277.50,578.00,108.288288,7000004,35.631560,M,Medium,M
7000010,12.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,24.0,...,Active,CUTTACK,12.00,60.00,400.000000,7000010,23.801064,L,Small,L
7000015,54.5,45.0,74.00,27.0,0.0,0.0,0.00,0.00,0.0,0.0,...,Inactive,PURI,200.50,0.00,-100.000000,7000015,16.091194,L,Medium,L
7000025,75.0,95.0,48.00,108.8,73.0,167.0,137.00,173.00,158.0,132.0,...,Active,PURI,566.80,856.00,51.023289,7000025,16.091194,M,Medium,M
7000027,513.0,5.0,171.95,207.5,92.0,68.5,35.00,144.95,0.0,0.0,...,Inactive,BARGARH,1057.95,179.95,-82.990690,7000027,-10.631377,M,Large,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036277,39.0,75.0,26.00,31.0,11.0,72.0,103.00,124.00,77.0,73.0,...,Active,KEONJHAR,254.00,492.00,93.700787,7036277,19.238393,M,Medium,M
7036279,67.0,65.0,30.00,61.0,6.0,22.5,54.00,70.00,17.0,14.0,...,Active,Kallakurichi,251.50,167.00,-33.598410,7036279,54.430631,M,Medium,M
7036287,89.0,104.0,111.00,122.0,137.0,100.0,111.00,146.00,200.0,207.0,...,Active,JAGATSINGHPUR,663.00,1012.00,52.639517,7036287,36.292578,M,Medium,M
7036291,62.5,75.0,60.00,96.0,108.0,132.0,168.00,178.00,180.0,236.5,...,Active,Nagpur,533.50,1040.50,95.032802,7036291,38.950880,M,Medium,H
